<div dir="rtl" style="font-family: 'Tahoma', 'Arial'; line-height: 1.8; text-align: justify; padding: 40px; border: 1px solid #d1d9e6; border-radius: 15px; background-color: #ffffff; color: #2c3e50; margin: 20px auto; max-width: 850px; box-shadow: 0 4px 20px rgba(0,0,0,0.05);">

<h2 style="color: #159895; border-bottom: 2px solid #1a5f7a; padding-bottom: 10px;">بخش ۱ از فاز ۲: پیاده‌سازی معماری مدل عمیق</h2>

<p>در این مرحله، پس از ارزیابی مدل Baseline در فاز اول، معماری نهایی سیستم بر پایه یادگیری عمیق طراحی شده است. انتخاب ما مدل <strong>BERT (Bidirectional Encoder Representations from Transformers)</strong> است که در حال حاضر استاندارد طلایی در پردازش زبان طبیعی محسوب می‌شود.</p>

<h3 style="color: #1a5f7a;">۱.۱. انتخاب مدل پیش‌آموزش‌دیده (Pre-trained Model)</h3>
<p>ما از نسخه <code>bert-base-uncased</code> استفاده می‌کنیم. این مدل بر روی حجم عظیمی از داده‌های ویکی‌پدیا و کتاب‌ها آموزش دیده و درک عمیقی از روابط معنایی کلمات دارد. برای انطباق با دیتاست Bitext، لایه نهایی این مدل با یک لایه Classification جدید جایگزین می‌شود که دارای ۲۷ خروجی (مطابق با تعداد Intentها) است.</p>

<h3 style="color: #1a5f7a;">۱.۲. طراحی کلاس Dataset تخصصی</h3>
<p>برای تغذیه داده‌ها به مدل در کتابخانه <strong>PyTorch</strong>، یک کلاس Custom Dataset طراحی شده است که وظایف زیر را بر عهده دارد:</p>
<ul style="padding-right: 25px;">
<li>فراخوانی Tokenizer اختصاصی BERT برای تبدیل متن به <code>input_ids</code> و <code>attention_mask</code>.</li>
<li>یکسان‌سازی طول جملات (Padding/Truncation) بر اساس تحلیل‌های بخش EDA (طول حداکثر ۶۴ توکن).</li>
<li>آماده‌سازی تنسورها (Tensors) برای پردازش توسط کارت گرافیک (GPU).</li>
</ul>

<h3 style="color: #1a5f7a;">۱.۳. ساختار لایه‌های نهایی</h3>
<p>خروجی لایه <code>[CLS]</code> مدل BERT به یک لایه <strong>Dropout</strong> (برای جلوگیری از بیش‌برازش) و سپس به یک لایه <strong>Linear</strong> متصل می‌شود تا احتمالات مربوط به هر یک از ۲۷ کلاس قصد (Intent) را محاسبه کند.</p>

</div>

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

class IntentDataset(torch.utils.data.Dataset):
    """کلاس سفارشی برای آماده‌سازی داده‌های Bitext برای مدل BERT"""
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

class BERTIntentClassifier(nn.Module):
    """معماری مدل نهایی بر پایه BERT برای تشخیص ۲۷ نوع Intent"""
    def __init__(self, n_classes):
        super(BERTIntentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.drop = nn.Dropout(p=0.3) # لایه دراپ‌اوت برای جلوگیری از اوورفیتینگ
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        # خروجی BERT شامل دو بخش است؛ ما از pooled_output استفاده می‌کنیم
        _, pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False
        )
        output = self.drop(pooled_output)
        return self.out(output)

# --- تنظیمات اولیه برای استفاده در بخش‌های بعد ---
def get_model_and_tokenizer(n_classes):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BERTIntentClassifier(n_classes)
    return model, tokenizer

<div dir="rtl" style="font-family: 'Tahoma', 'Arial'; line-height: 1.8; text-align: justify; padding: 40px; border: 1px solid #d1d9e6; border-radius: 15px; background-color: #ffffff; color: #2c3e50; margin: 20px auto; max-width: 850px; box-shadow: 0 4px 20px rgba(0,0,0,0.05);">

<h2 style="color: #159895; border-bottom: 2px solid #1a5f7a; padding-bottom: 10px;">بخش ۲ از فاز ۲: طراحی چرخه آموزش و بهینه‌سازی</h2>

<p>در این بخش، پروتکل آموزش مدل طراحی شده است. هدف ما در این مرحله، تنظیم وزن‌های لایه نهایی BERT به گونه‌ای است که کمترین میزان خطا (Loss) را در تشخیص ۲۷ قصد (Intent) دیتاست Bitext داشته باشد.</p>

<h3 style="color: #1a5f7a;">۲.۱. تابع هزینه و بهینه‌ساز (Loss & Optimizer)</h3>
<p>برای آموزش موثر، ابزارهای زیر انتخاب شده‌اند:</p>
<ul style="padding-right: 25px;">
<li><strong>Criterion (CrossEntropyLoss):</strong> از آنجایی که با یک مسئله دسته‌بندی چندکلاسه (Multi-class) روبرو هستیم، این تابع هزینه بهترین گزینه برای اندازه‌گیری فاصله بین پیش‌بینی مدل و برچسب واقعی است.</li>
<li><strong>Optimizer (AdamW):</strong> این بهینه‌ساز نسخه اصلاح‌شده Adam است که برای مدل‌های Transformer پیشنهاد می‌شود، زیرا با استفاده از Weight Decay بهتر می‌تواند از Overfitting جلوگیری کند.</li>
<li><strong>Scheduler:</strong> برای پایداری آموزش، از یک Linear Scheduler با Warmup استفاده می‌کنیم تا نرخ یادگیری (Learning Rate) به تدریج کاهش یابد.</li>
</ul>

<h3 style="color: #1a5f7a;">۲.۲. استراتژی جلوگیری از بیش‌برازش (Regularization)</h3>
<p>با توجه به حجم متوسط دیتاست (۲۷ هزار سطر)، احتمال Overfitting وجود دارد. راهکارهای ما عبارتند از:</p>
<ol>
<li><strong>Early Stopping:</strong> توقف آموزش در صورتی که دقت روی داده‌های Validation دیگر بهبود نیابد.</li>
<li><strong>Gradient Clipping:</strong> محدود کردن اندازه گرادیان‌ها برای جلوگیری از پدیده انفجار گرادیان (Exploding Gradients).</li>
</ol>

<h3 style="color: #1a5f7a;">۲.۳. طراحی تابع Train و Evaluate</h3>
<p>چرخه آموزش به گونه‌ای طراحی شده که در هر اپوک (Epoch)، مدل ابتدا روی داده‌های Train آموزش می‌بیند و سپس بلافاصله روی داده‌های Validation تست می‌شود تا سلامت فرآیند یادگیری مانیتور شود.</p>

</div>

In [ ]:
import torch
from torch.utils.data import DataLoader
import pandas as pd
from transformers import BertTokenizer

# ۱. وارد کردن ماژول‌هایی که در فایل‌های پایتون ساختیم
import sys
import os
sys.path.append(os.path.abspath('../../src'))
from models.architectures import BERTIntentClassifier, IntentDataset
from training.trainer import train_epoch, eval_model, setup_training

# ۲. تنظیمات هایپرپارامترها (طبق پلن فاز ۱)
MAX_LEN = 64
BATCH_SIZE = 32
EPOCHS = 5
LEARNING_RATE = 2e-5

# ۳. بارگذاری داده‌های تقسیم‌شده (خروجی فاز ۱)
train_df = pd.read_csv('../../data/processed/train.csv')
val_df = pd.read_csv('../../data/processed/val.csv')

# ۴. آماده‌سازی Tokenizer و DataLoaders
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_ds = IntentDataset(train_df['instruction'].to_numpy(), train_df['intent_encoded'].to_numpy(), tokenizer, MAX_LEN)
val_ds = IntentDataset(val_df['instruction'].to_numpy(), val_df['intent_encoded'].to_numpy(), tokenizer, MAX_LEN)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)

# ۵. مقداردهی اولیه مدل و ابزارهای آموزش
n_classes = train_df['intent_encoded'].nunique()
model = BERTIntentClassifier(n_classes)

device, optimizer, scheduler, loss_fn = setup_training(
    model, len(train_loader), EPOCHS, lr=LEARNING_RATE
)

print(f"🚀 آماده آموزش روی {device} هستیم...")
print(f"تعداد کل داده‌های آموزش: {len(train_df)}")

# این سلول فقط زیرساخت را آماده کرد. در بخش ۳ فاز ۲، حلقه اصلی را اجرا می‌کنیم.

<div dir="rtl" style="font-family: 'Tahoma', 'Arial'; line-height: 1.8; text-align: justify; padding: 40px; border: 1px solid #d1d9e6; border-radius: 15px; background-color: #ffffff; color: #2c3e50; margin: 20px auto; max-width: 850px; box-shadow: 0 4px 20px rgba(0,0,0,0.05);">

<h2 style="color: #159895; border-bottom: 2px solid #1a5f7a; padding-bottom: 10px;">بخش ۳ از فاز ۲: اجرای آموزش و مانیتورینگ (Training Execution)</h2>

<p>در این مرحله، مدل BERT بر روی ۲۷ هزار داده‌ی پیش‌پردازش شده‌ی Bitext آموزش داده می‌شود. فرآیند آموزش به صورت Epoch-based طراحی شده و در هر مرحله سلامت یادگیری مدل رصد می‌گردد.</p>

<h3 style="color: #1a5f7a;">۳.۱. فرآیند آموزش و اعتبارسنجی همزمان</h3>
<p>برای اطمینان از اینکه مدل دچار <strong>Overfitting</strong> (بیش‌برازش) نمی‌شود، پس از اتمام هر Epoch آموزش، مدل بلافاصله بر روی داده‌های Validation ارزیابی می‌شود. این کار به ما اجازه می‌دهد تا منحنی یادگیری مدل را تحلیل کنیم.</p>

<h3 style="color: #1a5f7a;">۳.۲. ذخیره‌سازی هوشمند (Model Checkpointing)</h3>
<p>ما از استراتژی <strong>Best Model Saving</strong> استفاده می‌کنیم. در پایان هر دور، اگر دقت مدل بر روی داده‌های اعتبارسنجی بهبود یافته باشد، وزن‌های مدل در پوشه <code>models/</code> با فرمت <code>.bin</code> یا <code>.pt</code> ذخیره می‌شوند. این کار تضمین می‌کند که نویزهای اپوک‌های آخر، کیفیت مدل نهایی را خراب نکنند.</p>

<h3 style="color: #1a5f7a;">۳.۳. تحلیل منحنی‌های یادگیری</h3>
<p>نمودارهای Loss و Accuracy که در انتهای این مرحله رسم می‌شوند، ابزار اصلی ما برای اثبات کیفیت آموزش به استاد هستند. انتظار می‌رود با گذشت اپوک‌ها، میزان Loss به تدریج کاهش و Accuracy به یک سطح پایدار (بالای ۹۰٪) برسد.</p>

</div>

In [ ]:
from collections import defaultdict
import matplotlib.pyplot as plt

history = defaultdict(list)
best_accuracy = 0

# حلقه اصلی آموزش
for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    # مرحله آموزش روی داده‌های Train
    train_acc, train_loss = train_epoch(
        model,
        train_loader,
        loss_fn,
        optimizer,
        device,
        scheduler,
        len(train_df)
    )
    print(f'Train loss {train_loss:.4f} accuracy {train_acc:.4f}')

    # مرحله اعتبارسنجی روی داده‌های Validation
    val_acc, val_loss = eval_model(
        model,
        val_loader,
        loss_fn,
        device,
        len(val_df)
    )
    print(f'Val   loss {val_loss:.4f} accuracy {val_acc:.4f}')
    print()

    # ذخیره تاریخچه برای رسم نمودار
    history['train_acc'].append(train_acc.item())
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc.item())
    history['val_loss'].append(val_loss)

    # ذخیره بهترین مدل بر اساس دقت Validation
    if val_acc > best_accuracy:
        torch.save(model.state_dict(), '../../models/best_model_state.bin')
        best_accuracy = val_acc

# --- رسم نمودارهای مانیتورینگ ---
plt.figure(figsize=(12, 5))

# نمودار Accuracy
plt.subplot(1, 2, 1)
plt.plot(history['train_acc'], label='train accuracy', color='#1a5f7a')
plt.plot(history['val_acc'], label='validation accuracy', color='#159895')
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

# نمودار Loss
plt.subplot(1, 2, 2)
plt.plot(history['train_loss'], label='train loss', color='#1a5f7a')
plt.plot(history['val_loss'], label='validation loss', color='#159895')
plt.title('Training and Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.tight_layout()
plt.savefig('../../results/charts/training_monitoring.png')
plt.show()

print(f"✅ آموزش به پایان رسید. بهترین دقت به دست آمده: {best_accuracy:.4f}")

<div dir="rtl" style="font-family: 'Tahoma', 'Arial'; line-height: 1.8; text-align: justify; padding: 40px; border: 1px solid #d1d9e6; border-radius: 15px; background-color: #ffffff; color: #2c3e50; margin: 20px auto; max-width: 850px; box-shadow: 0 4px 20px rgba(0,0,0,0.05);">

<h2 style="color: #159895; border-bottom: 2px solid #1a5f7a; padding-bottom: 10px;">بخش ۴ از فاز ۲: ارزیابی دقیق و تحلیل خطا (Evaluation)</h2>

<p>در این مرحله، عملکرد نهایی مدل بر روی داده‌های آزمون (Test Set) که شامل ۴,۱۰۰ نمونه دست‌نخورده است، سنجیده می‌شود. هدف ما فراتر از یک عدد ساده، درک رفتار مدل در مواجهه با ۲۷ قصد مختلف است.</p>

<h3 style="color: #1a5f7a;">۴.۱. تحلیل ماتریس اغتشاش (Confusion Matrix)</h3>
<p>ماتریس اغتشاش ابزار اصلی ما برای شناسایی نقاط کور مدل است. با توجه به تعداد زیاد کلاس‌ها، این ماتریس به ما نشان می‌دهد که کدام Intentها شباهت معنایی زیادی از دید مدل دارند (مثلاً تفاوت بین <code>cancel_order</code> و <code>check_refund_status</code>).</p>

<h3 style="color: #1a5f7a;">۴.۲. گزارش جامع طبقه‌بندی (Classification Report)</h3>
<p>برای هر یک از ۲۷ کلاس، معیارهای زیر به صورت جداگانه محاسبه می‌گردد:</p>
<ul style="padding-right: 25px;">
<li><strong>Precision:</strong> دقت مدل در زمانی که یک قصد را پیش‌بینی می‌کند.</li>
<li><strong>Recall:</strong> توانایی مدل در پیدا کردن تمام نمونه‌های مربوط به یک قصد خاص.</li>
<li><strong>F1-Score:</strong> میانگین هارمونیک دو معیار بالا که به عنوان شاخص نهایی موفقیت در این پروژه در نظر گرفته شده است.</li>
</ul>

<h3 style="color: #1a5f7a;">۴.۳. تحلیل کیفی خطاها</h3>
<p>در انتهای این بخش، نمونه‌هایی از جملاتی که مدل در تشخیص آن‌ها دچار خطا شده است را استخراج می‌کنیم. این تحلیل به ما کمک می‌کند تا بفهمیم آیا مشکل از کمبود داده در آن کلاس خاص است یا ابهام در متن ورودی کاربر.</p>

</div>

In [ ]:
import torch
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from torch.utils.data import DataLoader

# ۱. بارگذاری داده‌های تست و مدل ذخیره شده
test_df = pd.read_csv('../../data/processed/test.csv')
test_ds = IntentDataset(test_df['instruction'].to_numpy(), test_df['intent_encoded'].to_numpy(), tokenizer, MAX_LEN)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

# بارگذاری وزن‌های بهترین مدل
model.load_state_dict(torch.load('../../models/best_model_state.bin'))
model = model.to(device)

def get_predictions(model, data_loader):
    model = model.eval()
    instructions = []
    predictions = []
    prediction_probs = []
    real_values = []

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)

            instructions.extend(d["text"])
            predictions.extend(preds)
            prediction_probs.extend(outputs)
            real_values.extend(labels)

    predictions = torch.stack(predictions).cpu()
    real_values = torch.stack(real_values).cpu()
    return instructions, predictions, real_values

# ۲. اجرای پیش‌بینی روی داده‌های تست
y_texts, y_pred, y_test = get_predictions(model, test_loader)

# ۳. چاپ گزارش نهایی (Classification Report)
class_names = list(encoder.classes_) # نام‌های متنی کلاس‌ها
print("\n📊 Final Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=class_names))

# ۴. رسم ماتریس اغتشاش (Confusion Matrix)
cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)

plt.figure(figsize=(15, 12))
hmap = sns.heatmap(df_cm, annot=False, fmt="d", cmap="Blues")
hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
plt.ylabel('True Intent')
plt.xlabel('Predicted Intent')
plt.title('Confusion Matrix - Final BERT Model')
plt.savefig('../../results/charts/final_confusion_matrix.png')
plt.show()

<div dir="rtl" style="font-family: 'Tahoma', 'Arial'; line-height: 1.8; text-align: justify; padding: 40px; border: 1px solid #d1d9e6; border-radius: 15px; background-color: #ffffff; color: #2c3e50; margin: 20px auto; max-width: 850px; box-shadow: 0 4px 20px rgba(0,0,0,0.05);">

<h2 style="color: #159895; border-bottom: 2px solid #1a5f7a; padding-bottom: 10px;">بخش ۵ از فاز ۲: استخراج موجودیت و منطق پاسخگویی (Entity & Logic)</h2>

<p>در این مرحله، چت‌بات قابلیت استخراج اطلاعات کلیدی (Entities) را پیدا می‌کند. تشخیص قصد (Intent) به تنهایی کافی نیست؛ برای مثال اگر کاربر قصد "پیگیری سفارش" داشته باشد، سیستم باید بتواند "شماره سفارش" را نیز از پیام او استخراج کند.</p>

<h3 style="color: #1a5f7a;">۵.۱. استخراج موجودیت (Entity Extraction)</h3>
<p>با توجه به ساختار دیتاست Bitext که شامل متغیرهایی مثل شماره سفارش، تاریخ، و مبالغ در قالب <code>{{ ... }}</code> است، ما از ترکیبی از دو روش زیر استفاده می‌کنیم:</p>
<ul style="padding-right: 25px;">
<li><strong>Regular Expressions (Regex):</strong> برای استخراج الگوهای ثابت مثل شماره سفارش‌های عددی یا ایمیل‌ها.</li>
<li><strong>Rule-based Parsing:</strong> برای شناسایی کلمات کلیدی مرتبط با دسته‌بندی‌ها (مثل نام محصول یا نوع شکایت).</li>
</ul>

<h3 style="color: #1a5f7a;">۵.۲. طراحی مخزن پاسخ‌ها (Response Database)</h3>
<p>برای هر یک از ۲۷ قصد شناخته شده، یک الگو (Template) برای پاسخ‌دهی طراحی شده است. این پاسخ‌ها پویا هستند؛ یعنی اطلاعات استخراج شده در مرحله قبل، مستقیماً درون پاسخ نهایی جایگذاری می‌شوند تا تجربه کاربری شخصی‌سازی شده‌ای ارائه گردد.</p>

<h3 style="color: #1a5f7a;">۵.۳. الگوریتم تصمیم‌گیری (Decision Logic)</h3>
<p>منطق سیستم به این صورت عمل می‌کند:</p>
<ol>
<li>مدل BERT برچسب Intent را مشخص می‌کند.</li>
<li>اگر Intent نیاز به داده اضافی داشته باشد (مثلاً <code>check_order</code>)، استخراج‌کننده فراخوانی می‌شود.</li>
<li>در صورت نبود داده لازم، سیستم به صورت هوشمند از کاربر درخواست اطلاعات (Slot Filling) می‌کند.</li>
</ol>

</div>

In [ ]:
import re

class ChatbotResponseManager:
    def __init__(self, intent_names):
        self.intent_names = intent_names
        # تعریف الگوهای پاسخ برای هر Intent
        self.response_templates = {
            'get_order': "I found your order {entity}. It's currently being processed!",
            'get_refund': "I've initiated a refund request for {entity}. You'll receive an email shortly.",
            'cancel_order': "Order {entity} has been successfully cancelled.",
            'edit_account': "I can help you update your account details. What would you like to change?",
            'check_shipping_status': "The shipping status for your request is: In Transit.",
            'default': "I understand you want to: {intent}. How can I assist you further with this?"
        }

    def extract_entities(self, text):
        """استخراج شماره سفارش یا کدهای پیگیری با استفاده از الگوهای عددی"""
        # جستجو برای اعدادی که شبیه کد سفارش هستند (مثلاً کد 5 تا 10 رقمی)
        order_match = re.search(r'#?\d{5,10}', text)
        if order_match:
            return order_match.group()
        return "N/A"

    def generate_response(self, intent_idx, original_text):
        """تولید پاسخ نهایی بر اساس قصد تشخیص داده شده و موجودیت‌ها"""
        intent_name = self.intent_names[intent_idx]
        entity = self.extract_entities(original_text)
        
        # انتخاب قالب پاسخ
        template = self.response_templates.get(intent_name, self.response_templates['default'])
        
        # جایگذاری داده‌ها در قالب
        if "{entity}" in template:
            if entity == "N/A":
                return f"I understand you're asking about {intent_name.replace('_', ' ')}, but could you please provide your order number?"
            return template.format(entity=entity)
        
        return template.format(intent=intent_name.replace('_', ' '))

# --- مثال برای تست در نوت‌بوک ---
# manager = ChatbotResponseManager(encoder.classes_)
# print(manager.generate_response(5, "I want to cancel my order #55432"))

بسیار عالی! به ایستگاه پایانی پروژه رسیدیم. طبق مستندات پروژه (صفحه ۲۵ و بخش ارائه)، داشتن یک دموی زنده (Live Demo) یا رابط کاربری، نشان‌دهنده تکمیل چرخه توسعه و آماده بودن محصول برای دنیای واقعی است. این بخش بیشترین تاثیر را در ارائه نهایی به استاد دارد.

در ادامه، گزارش مارک‌داون و کد کامل رابط کاربری با کتابخانه Gradio (که ساده‌ترین و حرفه‌ای‌ترین ابزار برای دموی مدل‌های هوش مصنوعی است) را برایت آماده کرده‌ام.

<div dir="rtl" style="font-family: 'Tahoma', 'Arial'; line-height: 1.8; text-align: justify; padding: 40px; border: 1px solid #d1d9e6; border-radius: 15px; background-color: #ffffff; color: #2c3e50; margin: 20px auto; max-width: 850px; box-shadow: 0 4px 20px rgba(0,0,0,0.05);">

<h2 style="color: #159895; border-bottom: 2px solid #1a5f7a; padding-bottom: 10px;">بخش ۶ از فاز ۲: توسعه دمو و رابط کاربری (Deployment & Demo)</h2>

<p>در آخرین مرحله از پروژه، تمامی اجزای طراحی شده در فاز ۱ و ۲ (شامل مدل BERT، استخراج‌کننده موجودیت و مدیریت پاسخ‌ها) در قالب یک رابط کاربری گرافیکی (GUI) یکپارچه شدند.</p>

<h3 style="color: #1a5f7a;">۶.۱. طراحی رابط کاربری (UI)</h3>
<p>برای نمایش عملکرد چت‌بات، از کتابخانه <strong>Gradio</strong> استفاده شده است. این ابزار به ما اجازه می‌دهد بدون درگیری با کدهای پیچیده فرانت‌اند، یک محیط چت تعاملی بسازیم که قابلیت‌های زیر را دارد:</p>
<ul style="padding-right: 25px;">
<li>دریافت پیام متنی کاربر به صورت آنی.</li>
<li>نمایش Intent تشخیص داده شده (برای شفافیت عملکرد مدل).</li>
<li>ارائه پاسخ نهایی هوشمند همراه با اطلاعات استخراج شده.</li>
</ul>

<h3 style="color: #1a5f7a;">۶.۲. معماری سیستم در زمان اجرا (Inference Pipeline)</h3>
<p>هنگامی که کاربر پیامی ارسال می‌کند، مراحل زیر در کسری از ثانیه طی می‌شود:</p>
<ol>
<li><strong>Preprocessing:</strong> متن کاربر توکن‌بندی شده و به تنسور تبدیل می‌شود.</li>
<li><strong>Model Inference:</strong> مدل BERT آموزش‌دیده، احتمالات ۲۷ کلاس را محاسبه کرده و برچسب نهایی را می‌زند.</li>
<li><strong>Response Generation:</strong> با استفاده از کلاس <code>ResponseManager</code>، موجودیت‌ها استخراج شده و پاسخ نهایی ساخته می‌شود.</li>
</ol>

<h3 style="color: #1a5f7a;">۶.۳. آماده‌سازی برای تحویل نهایی</h3>
<p>ساختار نهایی پروژه طبق نمودار درختی ارائه شده در فاز ۱، شامل تمامی مدل‌های ذخیره شده، کدهای ماژولار و گزارش‌های ارزیابی است. پروژه آماده است تا با اجرای یک دستور، در محیط محلی یا سرورهای ابری (مثل HuggingFace Spaces) میزبانی شود.</p>

</div>